In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr

In [3]:
tfds.__version__

'4.5.2'

In [3]:
!echo $TFDS_DATA_DIR

/home/guhangsong/Data/tf_datasets


In [2]:
ds = tfds.load('mslr_web/30k_fold1', split='train')

2022-03-20 14:26:09.929303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-20 14:26:09.990809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-20 14:26:09.991251: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-20 14:26:09.994758: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow wi

In [3]:
# 增加_mask特征
ds = ds.map(lambda feature_map: {
    "_mask": tf.ones_like(feature_map["label"], dtype=tf.bool),
    **feature_map
})

In [4]:
# 数据补齐，
ds = ds.shuffle(buffer_size=1000).padded_batch(batch_size=32)

In [5]:
# 生成标签对，补齐数据的标签置为-1
ds = ds.map(lambda feature_map: (
    feature_map, tf.where(feature_map["_mask"], feature_map.pop("label"), -1.)))

In [6]:
# 这里的shape是忽略batch的单样本shape，最后的1表示一个特征，前面的None其实是query下的多个doc
inputs = {
    name: tf.keras.Input(shape=(None, 1), dtype=tf.float32, name=name)
    for name in ds.element_spec[0]
    if name != "_mask"
}
#batch正则化
norm_inputs = [tf.keras.layers.BatchNormalization()(x) for x in inputs.values()]

# 特征拼接
x = tf.concat(norm_inputs, axis=-1)
for layer_width in [128, 64, 32]:
  x = tf.keras.layers.Dense(units=layer_width)(x)
  x = tf.keras.layers.Activation(activation=tf.nn.relu)(x)

# 消除最外层tensor
scores = tf.squeeze(tf.keras.layers.Dense(units=1)(x), axis=-1)

In [7]:
model = tf.keras.Model(inputs=inputs, outputs=scores)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tfr.keras.losses.PairwiseLogisticLoss(),
    metrics=tfr.keras.metrics.get("ndcg", topn=10, name="NDCG@10"))
model.fit(ds, epochs=3)

Epoch 1/3


/opt/miniconda3/envs/ds/lib/python3.9/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['_mask'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


  1/592 [..............................] - ETA: 1:32:24 - loss: 12.4341 - NDCG@10: 0.1736

2022-03-20 14:26:33.199016: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


592/592 [==============================] - 53s 74ms/step - loss: 10.6635 - NDCG@10: 0.4021
Epoch 2/3
592/592 [==============================] - 43s 72ms/step - loss: 10.5344 - NDCG@10: 0.4233
Epoch 3/3
592/592 [==============================] - 43s 72ms/step - loss: 10.3554 - NDCG@10: 0.4303
